In [12]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib2
import re

In [2]:
#URL넣어주면 해당 박스스코어 파싱
class BoxScoreParser:
    def __init__(self,url):
        self.data = urllib2.urlopen(url)
        self.html = BeautifulSoup(self.data)
        
        #DATE
        self.data = self.html.select('div.yearDate span')[0].text
        
        #AWAY팀명
        self.a_Team=self.html.select('table.socreBoard tr:nth-of-type(2) th')[0].text
        
        #HOME팀명
        self.h_Team=self.html.select('table.socreBoard tr:nth-of-type(3) th')[0].text
        
        #AWAY_SCORE
        tmp=html.select('table.socreBoard tr:nth-of-type(2) td')
        self.a_Score=list()
        for score in tmp:
            self.a_Score.append(score.text)
        
        #HOME_SCORE
        tmp=html.select('table.socreBoard tr:nth-of-type(3) td')
        self.h_Score=list()
        for score in tmp:
            self.h_Score.append(score.text)
        
        #AWAY_STATUS
        self.a_status=html.select('p.results')[0].text
        
        #HOME_STATUS
        self.h_status=html.select('p.results')[1].text
        
        #STADIUM
        tmp=html.select('p.ballpark')[0]
        #.text 메서드는 unicode형태로 반환
        tmp=tmp.text
        self.stadium=tmp[tmp.find(u'고척'):tmp.find(u'관중')].strip()
        
        #HEAD
        keys=[u'결승타',u'홈런',u'3루타',u'2루타',u'실책',u'도루자',u'주루사',u'견제사',u'병살타',u'폭투',u'심판']
        self.head=dict()
        for k in keys:
            self.head[k]
        
        #AWAY 투수기록
        columns=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
        frame=html.select('table#xtable3 tbody:nth-of-type(1) tr')
        data=list()    #frame으로부터 데이터를 담을 리스트
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
                tmp.append(dat)
            data.append(tmp)   #정리된 list를 data에 추가
        self.a_pitRecord = pd.DataFrame(data=data,columns=columns)

        #HOME 투수기록
        #col=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
        frame=html.select('table#xtable3 tbody:nth-of-type(2) tr')
        data=list()    #frame으로부터 데이터를 담을 리스트
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
                tmp.append(dat)
            data.append(tmp)   #정리된 list를 data에 추가
        self.h_pitRecord = pd.DataFrame(data=data,columns=columns)
        #등판:선발의 선수명과 이닝,타자를 get
        # -> h_pitRecord[h_pitRecord[u'등판']==u'선발'].reindex(columns=[u'선수명',u'이닝',u'타자'])
        
        #AWAY_타자기록
        columns=[u'선수명',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9',u'10',u'11',u'12',u'타수',u'안타',u'타점',u'득점',u'타율']
        frame=html.select('table#xtable1 tbody:nth-of-type(1) tr')
        data=list()
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n')[2:]:
                tmp.append(dat)
            data.append(tmp)
        self.a_batRecord = pd.DataFrame(data=data,columns=columns)
        
        #HOME_타자기록
        columns=[u'선수명',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9',u'10',u'11',u'12',u'타수',u'안타',u'타점',u'득점',u'타율']
        frame=html.select('table#xtable1 tbody:nth-of-type(2) tr')
        data=list()
        for row in frame:
            tmp=list()
            for dat in row.text.strip().split('\n')[2:]:
                tmp.append(dat)
            data.append(tmp)
        self.h_batRecord = pd.DataFrame(data=data,columns=columns)

In [3]:
html=urllib2.urlopen('http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=1&gameId=20160327NCWO0&gyear=2016')
html=BeautifulSoup(html,'lxml')

In [4]:
#AWAY_타자기록
columns=[u'선수명',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9',u'10',u'11',u'12',u'타수',u'안타',u'타점',u'득점',u'타율']
frame=html.select('table#xtable1 tbody:nth-of-type(1) tr')
data=list()
for row in frame:
    tmp=list()
    for dat in row.text.strip().split('\n')[2:]:
        tmp.append(dat)
    data.append(tmp)
a = pd.DataFrame(data=data,columns=columns)

In [5]:
str('1234%d'%(6))

'12346'

In [6]:
b=dict()

In [7]:
b[1]=2

In [8]:
b

{1: 2}

In [9]:
keys=[u'결승타',u'홈런',u'3루타',u'2루타',u'실책',u'도루자',u'주루사',u'견제사',u'병살타',u'폭투',u'심판']
head=dict()
for num in range(11):
    frame=html.select('table.tEx tr:nth-of-type(%d) td'%(num+1))[0].text
    head[keys[num]]=frame
    
for k in head.keys():
    if k == u'심판':
        head[k]=head[k].strip().split()
    else:
        head[k]=head[k].strip().split(') ')    
        str.split

SyntaxError: invalid syntax (<ipython-input-9-84f0e6a34870>, line 4)

In [11]:
a='(ab c) (d ef) (g hi)'
re.sub

-1

In [145]:
'(ab c) (d ef) (g hi)'.split()

TypeError: expected a character buffer object

In [134]:
for k in head:
    print k
    for x in head[k]:
        print 1
        print x    
    print

심판
1
황인태
1
추평호
1
김준희
1
전일수

견제사
1
박정음(8회)

폭투
1
이재학(3회)

홈런
1
나성범3호(1회1점 박주현
1
박동원3호(2회2점 이재학
1
이호준4호(7회1점 마정길)

실책
1
이택근(3회)

3루타
1
김성욱(2회)

병살타
1
대니돈(5회
1
박동원(6회
1
서건창(7회)

결승타
1
나성범(1회 2사서 우월 홈런)

2루타
1
김태군(3회
1
강민국(9회)

주루사
1
박석민(3회
1
이재율(5회)

도루자
1
이택근(1회)



In [83]:
print html.select('table.tEx tr:nth-of-type(%d)'%(0+1))

[<tr>\n<th scope="row">\uacb0\uc2b9\ud0c0</th>\n<td>\ub098\uc131\ubc94(1\ud68c 2\uc0ac\uc11c \uc6b0\uc6d4 \ud648\ub7f0)</td>\n</tr>]


In [5]:
h_pitRecord

,선수명,등판,결과,승,패,세,이닝,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점
0,박주현,선발,패,0,1,0,2 2/3,18,63,18,11,1,0,3,7,7,7.27
1,김상수,3.1,,1,1,0,3 1/3,13,57,12,3,0,1,3,0,0,1.38
2,마정길,7.5,,0,0,0,1,4,9,4,1,1,0,0,1,1,2.57
3,김택형,8.9,,0,0,0,1,3,14,3,0,0,0,0,0,0,0.00
4,이보근,9.3,,0,0,0,1,5,19,5,2,0,0,1,0,0,1.80


In [43]:
#등판:선발의 선수명과 이닝,타자를 get
h_pitRecord[h_pitRecord[u'등판']==u'선발'].reindex(columns=[u'선수명',u'이닝',u'타자'])

,선수명,이닝,타자
0,박주현,2 2/3,18


In [42]:
h_pitRecord[h_pitRecord[u'등판']==u'선발'].reindex(columns=[u'선수명',u'이닝',u'타자'])[u'타자'][0]

u'18'

In [82]:
print a.index.tolist()
print a[0]

[0, 1, 2, 3, 4]
True


In [56]:
type(h_pitRecord[u'등판']==u'선발')

pandas.core.series.Series

In [101]:
a=h_pitRecord.select(lambda x: x==u'등판',axis=1)

In [ ]:
a_pitRecord.ge

In [8]:
print date
print a_Team
print h_Team
print a_Score
print h_Score
print stadium
print len(stadium)

2016.03.27(일)
NC
넥센
[u'1', u'3', u'3', u'0', u'0', u'0', u'1', u'0', u'0', u'-', u'-', u'-', u'8', u'17', u'0', u'1']
[u'0', u'2', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'-', u'-', u'-', u'3', u'8', u'1', u'3']
고척
2


In [140]:
print len(u'고현영')
print len('고현영')

3
9


In [ ]:
print tmp.find(u'고척')

In [126]:
print stadium


        구장 : 고척      
        관중 : 3,476      
        개시 : 13:00      
        종료 : 15:44      
        경기시간 : 2:44
    
